In [4]:
import pandas as pd
from itertools import count
import re

# === 1. 엑셀 파일 불러오기 ===
excel_path = "/Users/songhune/Downloads/gwashi.xlsx"
xls = pd.ExcelFile(excel_path)
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

# === 2. 주요 텍스트 중심 엔티티 시트 정의 ===
entity_sheets = ['Question', 'Exam(소대과)', 'Exam2(기타과거포함)']

# === 3. surrogate ID 매핑 ===
id_generator = (f"Q{str(i).zfill(6)}" for i in count(1))
legacy_to_surrogate = {}

for sheet in entity_sheets:
    df = sheets[sheet]
    if 'id' in df.columns:
        for legacy_id in df['id'].dropna().unique():
            if legacy_id not in legacy_to_surrogate:
                legacy_to_surrogate[legacy_id] = next(id_generator)

# === 4. 빈 행 제거 함수 ===
def drop_empty_rows(df, core_columns=None):
    if core_columns is None:
        core_columns = ['id', 'contents', 'abstract', 'category', 'source']

    # 실제 존재하는 컬럼만 사용
    core_columns_present = [col for col in core_columns if col in df.columns]

    # 공백만 포함된 셀은 NaN으로 간주
    df_cleaned = df.copy().applymap(lambda x: None if isinstance(x, str) and x.strip() == "" else x)

    # 지정 컬럼이 모두 NaN이면 제거
    filtered = df_cleaned.dropna(subset=core_columns_present, how='all')
    return filtered

# === 5. 각 시트에 surrogate ID 적용 및 빈 행 제거 ===
for sheet in entity_sheets:
    df = sheets[sheet]
    df = drop_empty_rows(df)
    if 'id' in df.columns:
        df['legacy_id'] = df['id']
        df['id'] = df['id'].map(legacy_to_surrogate)
    sheets[sheet] = df

# === 6. 엣지 시트에서 ID 매핑 적용 (빈 행 제거 X) ===
for sheet in xls.sheet_names:
    if sheet.startswith("Edge"):
        df = sheets[sheet].copy()
        for col in ['subject', 'object']:
            if col in df.columns:
                df[col] = df[col].map(legacy_to_surrogate).fillna(df[col])
        sheets[sheet] = df

# === 7. ID 매핑 테이블 추가 ===
mapping_df = pd.DataFrame.from_dict(legacy_to_surrogate, orient='index', columns=['id'])
mapping_df.index.name = 'legacy_id'
sheets['id_mapping'] = mapping_df

# === 8. 결과 저장 ===
output_path = "gwashi_cleaned.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ 정제 완료. 저장 위치: {output_path}")

/var/folders/57/27t8gd7n5z31s0wkt5qkb98r0000gn/T/ipykernel_58679/1527292973.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.copy().applymap(lambda x: None if isinstance(x, str) and x.strip() == "" else x)
/var/folders/57/27t8gd7n5z31s0wkt5qkb98r0000gn/T/ipykernel_58679/1527292973.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.copy().applymap(lambda x: None if isinstance(x, str) and x.strip() == "" else x)
/var/folders/57/27t8gd7n5z31s0wkt5qkb98r0000gn/T/ipykernel_58679/1527292973.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.copy().applymap(lambda x: None if isinstance(x, str) and x.strip() == "" else x)


✅ 정제 완료. 저장 위치: gwashi_cleaned.xlsx


In [7]:
import pandas as pd

# 원본 파일 불러오기
excel_path = "/Users/songhune/Downloads/gwashi.xlsx"
xls = pd.ExcelFile(excel_path)
all_sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

# 병합된 exam 만들기 (앞서 만든 함수 기반)
exam1 = all_sheets.get('Exam(소대과)', pd.DataFrame()).copy()
exam2 = all_sheets.get('Exam2(기타과거포함)', pd.DataFrame()).copy()
merged = pd.merge(exam1, exam2, on='id', how='outer', suffixes=('_exam1', '_exam2'))

# 컬럼 병합
def coalesce(row, col): return row.get(f"{col}_exam2") if pd.notna(row.get(f"{col}_exam2")) else row.get(f"{col}_exam1")
candidate_columns = {col.replace('_exam1', '') for col in merged.columns if '_exam1' in col} | \
                    {col.replace('_exam2', '') for col in merged.columns if '_exam2' in col}
for col in candidate_columns:
    merged[col] = merged.apply(lambda r: coalesce(r, col), axis=1)
merged_exam = merged[['id'] + sorted(candidate_columns)]

# 최종 시트 구성
essential_sheets = {
    'Question': all_sheets['Question'],
    'Edge(Q-E)': all_sheets['Edge(Q-E)'],
    'merged_exam': merged_exam
}

# 저장
with pd.ExcelWriter("gwashi_minimal.xlsx", engine='openpyxl') as writer:
    for sheet_name, df in essential_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ 필수 시트만 포함된 gwashi_minimal.xlsx 생성 완료.")

✅ 필수 시트만 포함된 gwashi_minimal.xlsx 생성 완료.


In [8]:
# 1. 엑셀 파일 불러오기
file_path = "./gwashi_minimal.xlsx"
xls = pd.ExcelFile(file_path)
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

# 2. 각 시트 로드
questions = sheets['Question']
exams = sheets['merged_exam']
edge_qe = sheets['Edge(Q-E)']

# 3. Edge(Q-E) 컬럼명 표준화
edge_qe = edge_qe.rename(columns={
    'source id': 'question_id',
    'target id': 'exam_id'
})

# 4. 병합: Question ↔ Edge(Q-E)
merged = questions.merge(edge_qe, left_on='id', right_on='question_id', how='left')

# 5. 병합: + Exam 정보
merged_df = merged.merge(exams, left_on='exam_id', right_on='id', how='left', suffixes=('_question', '_exam'))

# 6. 결과 저장 (선택)
merged_df.to_excel("gwashi_merged_qe.xlsx", index=False)

print("✅ 병합 완료: 'gwashi_merged_qe.xlsx'로 저장되었습니다.")

✅ 병합 완료: 'gwashi_merged_qe.xlsx'로 저장되었습니다.


In [16]:

from korean_lunar_calendar import KoreanLunarCalendar
calendar = KoreanLunarCalendar()

# params : year(년), month(월), day(일)
calendar.setSolarDate(2017, 6, 24)

# Lunar Date (ISO Format)
print(calendar.LunarIsoFormat())

# Korean GapJa String
print(calendar.getGapJaString())

# Chinese GapJa String
print(calendar.getChineseGapJaString())

2017-05-01 Intercalation
정유년 병오월 임오일 (윤월)
丁酉年 丙午月 壬午日 (閏月)


In [ ]:
import pandas as pd
from korean_lunar_calendar import KoreanLunarCalendar

# 엑셀 파일 경로
file_path = "./gwashi_merged_qe.xlsx"

# 파일 로딩
df = pd.read_excel(file_path)

# 1. question_id에서 서력 날짜 추출
def extract_date(qid):
    try:
        if isinstance(qid, str) and qid.startswith('Q') and len(qid) >= 9:
            return int(qid[1:5]), int(qid[5:7]), int(qid[7:9])
    except:
        return None, None, None
    return None, None, None

df[['year', 'month', 'day']] = df['question_id'].apply(lambda x: pd.Series(extract_date(x)))

# 2. 간지 + 윤달 추출
def get_lunar_info(y, m, d):
    try:
        cal = KoreanLunarCalendar()
        cal.setSolarDate(y, m, d)
        return (
            cal.LunarIsoFormat(),            # e.g. '1795-07-12'
            cal.getGapJaString(),           # e.g. '을묘(년) 경신(월) 병술(일)'
            cal.getChineseGapJaString(),    # e.g. '乙卯(年) 庚申(月) 丙戌(日)'
            cal.isIntercalation()           # 윤달 여부
        )
    except:
        return None, None, None, None

df[['lunar_date', 'ganji_kr', 'ganji_cn', 'is_leap_month']] = df.apply(
    lambda row: pd.Series(get_lunar_info(row['year'], row['month'], row['day'])), axis=1
)

# 3. 저장
df.to_excel("gwashi_merged_with_ganji.xlsx", index=False)
print("✅ 간지 및 윤달 정보가 추가된 파일이 저장되었습니다: gwashi_merged_with_ganji.xlsx")

✅ 간지 및 윤달 정보 추가 완료. 파일 저장됨: gwashi_qe_with_ganji.xlsx


In [23]:
from korean_lunar_calendar import KoreanLunarCalendar
import pandas as pd
import re

# 1. 파일 로드
df = pd.read_excel("./gwashi_merged_qe.xlsx")

# 2. 날짜 파싱
def extract_date(qid):
    try:
        if isinstance(qid, str) and qid.startswith('Q') and len(qid) >= 9:
            return int(qid[1:5]), int(qid[5:7]), int(qid[7:9])
    except Exception as e:
        print(f"❌ 날짜 추출 실패: {qid} - {e}")
    return None, None, None

df[['year', 'month', 'day']] = df['question_id'].apply(lambda x: pd.Series(extract_date(x)))

# 3. 간지 및 윤달 여부 추출
def get_lunar_info(y, m, d):
    try:
        if pd.isna(y) or pd.isna(m) or pd.isna(d):
            return None, None, None, None, None, None, None, None
        y, m, d = int(y), int(m), int(d)
        cal = KoreanLunarCalendar()
        cal.setSolarDate(y, m, d)

        lunar_date = cal.LunarIsoFormat()
        ganji_kr = cal.getGapJaString()              # 기해년 정축월 기축일
        ganji_cn = cal.getChineseGapJaString()       # 己亥年 丁丑月 己丑日 (閏月)

        # 한국식 간지 분리
        ganji_kr_parts = ganji_kr.strip().split()
        kr_year = ganji_kr_parts[0].replace('년', '') if len(ganji_kr_parts) > 0 else None
        kr_month = ganji_kr_parts[1].replace('월', '') if len(ganji_kr_parts) > 1 else None
        kr_day = ganji_kr_parts[2].replace('일', '') if len(ganji_kr_parts) > 2 else None

        # 중국식 간지 분리 + 윤달 여부
        is_leap = '閏月' in ganji_cn
        match = re.search(r'(\S{2})年\s+(\S{2})月\s+(\S{2})日', ganji_cn)
        if match:
            cn_year, cn_month, cn_day = match.groups()
        else:
            cn_year, cn_month, cn_day = None, None, None

        return lunar_date, kr_year, kr_month, kr_day, cn_year, cn_month, cn_day, is_leap

    except Exception as e:
        print(f"❌ 간지 변환 실패: {y}-{m}-{d} - {e}")
        return None, None, None, None, None, None, None, None

# 4. 적용
df[['lunar_date', 'ganji_kr_year', 'ganji_kr_month', 'ganji_kr_day',
    'ganji_cn_year', 'ganji_cn_month', 'ganji_cn_day', 'is_leap_month']] = df.apply(
    lambda row: pd.Series(get_lunar_info(row['year'], row['month'], row['day'])), axis=1
)

# 5. 저장
df.to_excel("gwashi_debug_lunar_split.xlsx", index=False)
print("✅ 간지 및 윤달 정보 포함 파일 저장 완료: gwashi_debug_lunar_split.xlsx")

✅ 간지 및 윤달 정보 포함 파일 저장 완료: gwashi_debug_lunar_split.xlsx


In [18]:
from korean_lunar_calendar import KoreanLunarCalendar

calendar = KoreanLunarCalendar()
calendar.setSolarDate(1795, 8, 19)

print(calendar.LunarIsoFormat())           # 예: 1795-07-12
print(calendar.getGapJaString())           # 예: 을묘 경신 병술
print(calendar.getChineseGapJaString())    # 예: 乙卯 庚申 丙戌
print(calendar.isIntercalation())          # 예: False

1795-07-05
을묘년 갑신월 갑인일
乙卯年 甲申月 甲寅日


TypeError: 'bool' object is not callable